In [2]:
import blocksci

import sys, os, os.path, socket
import numpy as np
import zarr
import time
import pandas as pd

# Counting number of True and False values in cluster_is_black_ground_truth 

In [4]:
array_zarr = zarr.load('uniform_black/heur_1_data/cluster_is_black_ground_truth.zarr') #load zarr file into array
#df = pd.DataFrame (array_zarr) ## convert your array into a dataframe
#filepath = 'zarr_to_excel.csv'
#df.to_csv(filepath, index=False)
countTrue = np.where(array_zarr==True)
countFalse = np.where(array_zarr==False)
element_count_False = countFalse[0].size
element_count_True = countTrue[0].size
print(len(array_zarr))
print(element_count_False)
print(element_count_True)
print(array_zarr)

591692716
591692581
135
[False False False ... False False False]


# Counting number of True and False values in cluster_is_black_ground_truth 

In [6]:
array_zarr = zarr.load('uniform_black/heur_1_data/cluster_is_black_when_block.zarr') #load zarr file into array
df = pd.DataFrame (array_zarr) ## convert your array into a dataframe
filepath = 'zarr_to_excel.csv'
df.to_csv(filepath, index=False)
# countTrue = np.where(array_zarr==True)
# countFalse = np.where(array_zarr==False)
# element_count_False = countFalse[0].size
# element_count_True = countTrue[0].size
# print(len(array_zarr))
# print(element_count_False)
# print(element_count_True)
# print(array_zarr)

KeyboardInterrupt: 

# Gray Scale Model

### Important Variables:

In this task we will be implementing the SIR model with the following system parameters:

*   $a_{i}(b)$: total amount of assets hold by user i by the end of block b.
*   $d_{i}(b)$: total amount of black assets hold by node i by the end of block b.
*   $x_{i}(b)$: = $\frac{d_{i}(b)}{a_{i}(b)} $ black assets fraction by the end of block b.
*   $c_{i}(b)$: amount of clean asset owned by node i by the end of block b
*   $m_{i}(b)$: the amount mined by user i by the end of block b
*   $W(b)$: = {$w_{ij}$}$_{i,j=1,...,N}$ is the adjacency matrix of the transaction network at block b. wij is equal to the amount of bitcoin moved from user i to user j at block b.
*   $N = Nusers = |Nusers|$ total number of users in the network.

Below is a summary of the dynamics of the SIR model: